
**Flow Sensor Workshop- Part 1, Valler Vidergående Skole, 20 March 2019** <p>
_Sagar Sen, Senior Research Scientist, Simula Research Laboratory_
- What is breathing?
- Measurement of breathing using respiratory inductance plethysmography
- Reading _raw breathing data_ from the __[Flow sensor](http://www.sweetzpot.com/flow)__
- Visualizing breathing data using __[PlotLy](http://www.plot.ly)__  
- Exercise Set #1
- Find derivative of breathing data
- Defining breathing function $f(t)$
- Defining derivative of breathing function $f(t)'$
- The _bisection method_ to find roots
- The _Newton's method_ to find roots
- Visualize the roots and the signals
- Reflection points
- Exercise Set #2
   

**What is breathing?**

_"Breathing is the process of taking air into and expelling it from the lungs."_

<img src="./images/Diaphragmatic_breathing.gif" alt="Alt text that describes the graphic" title="Title text" height="200" width="200" />

**Measurment of breathing using Respiratory Inductance Plethysmography** <p>
_Respiratory inductance plethysmography (RIP) is a method of evaluating pulmonary ventilation by measuring the movement of the chest and abdominal wall._
 <img src="./images/rip.gif" alt="Respiratory Inductance Plethysmography" title="RIP" height="300" width="300"/>
    
The Flow sensor developed by __[Sweetzpot AS](http://www.sweetzpot.com)__ measures breathing from the _expansion_ and _contraction_ of the inspiratory muscles (chest and/or abdomen) and transmits the raw breathing data in _millivolts_ of potential difference across a semi-conductor strain gauge as shown in the figure below. 
<p> 
 <img src="./images/sensorInnerWorking.png" alt="Inner Working of the FLOW Sensor" title="Flow Sensor" class="center" height="500" width="500" />
<p>
The trasmission occurs wirelessly using Bluetooth 4.0 to a mobile phone App or to a USB Dongle.


**How is the strain measured?**

The strain is measured using a wheatstone's bridge network as shown in the figure below.

 <img src="./images/wheatStoneBridge.png" alt="Wheatstone's bridge" title="Title text" class="center" height="500" width="500"/>
 
We calculate the voltage in an _unbalanced wheatstone's bridge_ where the strain across the strain gauge changes with strectching.

$$V_{out} = V_C - V_D $$
$$V_{out} = \dfrac{R_2}{(R_1+R_2)} \times V_s - \dfrac{R_x}{(R_3 + R_4)} \times V_s$$

We measure $V_{out}$ using a multimeter in an integrated circuit for different values of $R_x$.
The different values of $V_{out}$ correspond to different values of the strain across the inspiratory muscles.

The analog voltage is further converted to a digital value between 0 and 4096 mV using an intergrated circuit multimeter. The strain is sampled 50 times per second, average over 5 consecutive values to give an output frequency of 10 times per second (10 Hz). These values are transmitted via Bluetooth packets of 20 bytes per second to a receiver on the desktop, or a mobile phone. The data is read and stored in a _comma separated value (csv)_ file.

**Reading breathing data**
- Specify breathing data filename
- Import __[Logging Library](https://docs.python.org/2/howto/logging.html)__ to log errors in the program
- Import __[Pandas Data Analysis Library](https://pandas.pydata.org/)__ to open and manipulate large datasets
- Read Comma Separated Value (CSV) file of breathing data from the Flow sensor
- Create Pandas Dataframe from breathing data

In [288]:
#Where is the breathing data located?
filename = "./BreathingData/sample.csv"

In [289]:

#Import Logging Library
import logging
logging.basicConfig(filename='flowTutorial.log',level=logging.DEBUG)

#Import Pandas Dataframe Library
try:
    import pandas as pd
    logging.info('Panadas library was successfully imported.')
except ImportError:
    logging.error("Pandas library was not found. Try pip install pandas")
    

try:
    breathingData =  pd.read_csv(filename)
    #Breathing data contains timestamp, relative time from start which is 0s , value of raw breathing signal, other irrelevant data
    breathingData.columns=['timestamp','reltime','value','other']
    print("Breathing Data Head Snippet...\n")
    #Print the first few lines of the breathing data
    print(breathingData.head())
    logging.info('File was opened and breathing data is copied into a Pandas dataframe')
except IOError:
    logging.error('Could not open file with breathing data')


Breathing Data Head Snippet...

      timestamp  reltime  value  other
0  1.547989e+09      0.1   1566    700
1  1.547989e+09      0.2   1565    700
2  1.547989e+09      0.3   1560    700
3  1.547989e+09      0.4   1560    700
4  1.547989e+09      0.5   1560    700


**Visualize breathing data**
- Import __[PlotLy](http://www.plot.ly)__ 
- Import __[Cufflinks](https://plot.ly/ipython-notebooks/cufflinks/)__ 
- Plot Raw Breathing Data

In [306]:
#Importing PlotLy and Cufflinks
try: 
    from plotly import __version__
    from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
    import plotly.plotly as py
    import plotly.graph_objs as go
    init_notebook_mode(connected=True)
except ImportError:
        logging.error("PlotLy or Cufflinks may not be installed.")

In [307]:
#Plot Breathing Data

traceRawBreathing = go.Scatter(x=breathingData.reltime,y=breathingData.value,name="Raw Breathing Signal")

#Specify Plot Layout
layout = go.Layout(
    
    title='Raw Breathing vs. Time',
    xaxis=dict(
        title='Time (s)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            color='black'
        ),
        showticklabels=True,
        tickangle=45,
        dtick=2,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        title='Voltage (mV)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            color='black'
        ),
        showticklabels=True,
        tickangle=45,
        dtick=100,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    )
)
fig = go.Figure(data=[traceRawBreathing], layout=layout)
iplot(fig, filename='axes-labels')


**Exercise Set #1**

1. Change the color of the line graph to red
2. Change the tick distance to 50 on the x-axis
3. Isolate a breath (inhale to exhale) of your choice and find the difference between the maximum value of strain at inhale and minimum value at exhale (manually)
4. Find the time between the an inhale and an exhale of your choice (manually)
5. Change tick angle to 0 degrees on both x and y axes

**Normalizing the breathing signal**

We would like to normalize the breathing signal based on the mean value of the breathing data.

- Import NumPy to compute the mean value and standard deviation of breathing signal
- We normalize the breathing signal array using the __[Standard Score](https://en.wikipedia.org/wiki/Standard_score)__
<p>
$$NormalizedBreathing[i] = \dfrac{Breathing[i]- mean(Breathing)}{std(Breathing)}$$
- Normalize the breathing values to z-score (-3 standard deviations to +3 standard deviations).
  The z-score is a number that shows how far a value is deviating from the mean.
- Visualize the normalized breathing



In [308]:
#Import Numpy
import numpy as np

def normalize(breathing):
    mean = np.mean(breathing)
    std = np.std(breathing)
    normalizedBreathing =[]
    i=0
    while i < len(breathing):
        normalizedBreathing.append((breathing[i]-mean)/np.std(breathing))
        i+=1
    return normalizedBreathing

In [309]:
#Normalize the breathing signal
normalized_breathingData = normalize(breathingData.value)
breathingData['normalizedValue'] = normalized_breathingData 


In [310]:
#Plot Normalized Breathing

traceNormalizedBreathing = go.Scatter(x=breathingData.reltime,y=breathingData.normalizedValue,name="Normalized Breathing Signal")

#Specify Plot Layout
layout = go.Layout(
    
    title='Normalized Breathing vs. Time',
    xaxis=dict(
        title='Time (s)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            color='black'
        ),
        showticklabels=True,
        tickangle=45,
        dtick=2,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        title='z-score',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            color='black'
        ),
        showticklabels=True,
        tickangle=45,
        dtick=0.5,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    )
)
fig = go.Figure(data=[traceNormalizedBreathing], layout=layout)
iplot(fig, filename='axes-labels')


**Finding the derivative of normalized breathing**

We need to now calculate the derivative of the breathing signal and visualize it.


In [311]:
#Derivative of the breathing signal

derivative=[]


try:
    for i in range(len(breathingData)-1):
        dT=breathingData.reltime[i+1]-breathingData.reltime[i]
        dV=breathingData.normalizedValue[i+1]-breathingData.normalizedValue[i]
        derivative.append(dV/dT)
except KeyError:
        logging.error("The index is incorrect.")
    
derivative.append(0)


breathingData['derivativeValue'] = derivative
    
    

**Visualize derivative of normalized breathing**

We now plot the derivative of the breathing signal.

In [312]:

#Plot Derivative Breathing

traceDerivativeBreathing = go.Scatter(x=breathingData.reltime,y=breathingData.derivativeValue,name="Derivative of Breathing Signal")

#Specify Plot Layout
layout = go.Layout(
    
    title='Derivative Breathing vs. Time',
    xaxis=dict(
        title='Time (s)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            color='black'
        ),
        showticklabels=True,
        tickangle=45,
        dtick=2,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        title='dz/dt',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            color='black'
        ),
        showticklabels=True,
        tickangle=45,
        dtick=2,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    )
)
fig = go.Figure(data=[traceDerivativeBreathing], layout=layout)
iplot(fig, filename='axes-labels')


**Finding the roots of the breathing function**


We do not really have a general  breathing function. Instead, we have a _table of values_ for the function $f(t)$ in time $t$. The roots of the function $f(t)$ are basically where the value of $f(t)$ crosses the zero of the x-axis.
<p>
Why would it be useful to find the roots? The roots are essentially when a transition happens from inhale to exhale and vice versa. A practical application of finding the root is to be able to compute the breathing rate in breaths per minute.

We will try to find the roots of the breathing signal using two techniques:
- Bisection method or Halving method
- Newton's method
    

In [313]:
#Define the breathing function from the lookup table
def f(x):
    errorMargin=0.0001
    functionVal=0
    for i in range(len(breathingData.reltime)):
        if abs(breathingData.reltime[i]-x)<=errorMargin:
            functionVal=breathingData.nor malizedValue[i] 
    return functionVal

In [314]:
#Define the derivative of the breathing function from the lookup table
def Df(x):
    errorMargin=0.0001
    minValue=9999
    functionVal=0
    for i in range(len(breathingData.reltime)):
        if abs(breathingData.reltime[i]-x)<=errorMargin:
            functionVal=breathingData.derivativeValue[i] 
    return functionVal       

In [315]:
def bisection(f,a,b,N):
    '''Approximate solution of f(x)=0 on interval [a,b] by the bisection method.

    Parameters
    ----------
    f : function
        The function for which we are trying to approximate a solution f(x)=0.
    a,b : numbers
        The interval in which to search for a solution. The function returns
        None if f(a)*f(b) >= 0 since a solution is not guaranteed.
    N : (positive) integer
        The number of iterations to implement.

    Returns
    -------
    x_N : number
        The midpoint of the Nth interval computed by the bisection method. The
        initial interval [a_0,b_0] is given by [a,b]. If f(m_n) == 0 for some
        midpoint m_n = (a_n + b_n)/2, then the function returns this solution.
        If all signs of values f(a_n), f(b_n) and f(m_n) are the same at any
        iteration, the bisection method fails and return None.

    Examples
    --------
    >>> f = lambda x: x**2 - x - 1
    >>> bisection(f,1,2,25)
    1.618033990263939
    >>> f = lambda x: (2*x - 1)*(x - 3)
    >>> bisection(f,0,1,10)
    0.5
    '''
    if f(a)*f(b) >= 0:
        print("Bisection method fails.")
        return None
    a_n = a
    b_n = b
    for n in range(1,N+1):
        m_n = (a_n + b_n)/2
        f_m_n = f(m_n)
        if f(a_n)*f_m_n < 0:
            a_n = a_n
            b_n = m_n
        elif f(b_n)*f_m_n < 0:
            a_n = m_n
            b_n = b_n
        elif f_m_n == 0:
            print("Found exact solution.")
            return m_n
        else:
            print("Bisection method fails.")
            return None
    return (a_n + b_n)/2

In [316]:
#Finding roots using the Newton's method

def newton(f,Df,x0,epsilon,max_iter):
    '''Approximate solution of f(x)=0 by Newton's method.

    Parameters
    ----------
    f : function
        Function for which we are searching for a solution f(x)=0.
    Df : function
        Derivative of f(x).
    x0 : number
        Initial guess for a solution f(x)=0.
    epsilon : number
        Stopping criteria is abs(f(x)) < epsilon.
    max_iter : integer
        Maximum number of iterations of Newton's method.

    Returns
    -------
    xn : number
        Implement Newton's method: compute the linear approximation
        of f(x) at xn and find x intercept by the formula
            x = xn - f(xn)/Df(xn)
        Continue until abs(f(xn)) < epsilon and return xn.
        If Df(xn) == 0, return None. If the number of iterations
        exceeds max_iter, then return None.

    Examples
    --------
    >>> f = lambda x: x**2 - x - 1
    >>> Df = lambda x: 2*x - 1
    >>> newton(f,Df,1,1e-8,10)
    Found solution after 5 iterations.
    1.618033988749989
    '''
    xn = x0
    for n in range(0,max_iter):
        fxn = f(xn)
        if abs(fxn) < epsilon:
            print('Found solution after',n,'iterations.')
            return xn
        Dfxn = Df(xn)
        if Dfxn == 0:
            print('Zero derivative. No solution found.')
            return None
        xn = xn - fxn/Dfxn
    print('Exceeded maximum iterations. No solution found.')
    return None

In [317]:
#Finding root using bisection method
startTime=3
endTime=10
bisection_root = bisection(f,startTime,endTime,100)

Found exact solution.


In [318]:
#Finding root using Newton's method
initialTime = 4
newton_root = newton(f,Df,initialTime,1e-10,100)

Found solution after 1 iterations.


**Plot Root of the Breathing Function**

We plot the roots found using: <p>
- Bisection method
- Newton's method

In [319]:
#Plot Derivative Breathing

traceNewtonRootBreathing = go.Scatter(x=[newton_root],y=[0],name="Newtwon's Method Root of Breathing Signal")
traceBisectionRootBreathing = go.Scatter(x=[bisection_root],y=[0],name="Bisection Method Root of Breathing Signal")

#Specify Plot Layout
layout = go.Layout(
    
    title='Numerical Analysis of Breathing',
    xaxis=dict(
        title='Time (s)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            color='black'
        ),
        showticklabels=True,
        tickangle=45,
        dtick=2,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    ),
    yaxis=dict(
        title='dz/dt',
        titlefont=dict(
            family='Arial, sans-serif',
            size=18,
            color='black'
        ),
        showticklabels=True,
        tickangle=45,
        dtick=2,
        tickfont=dict(
            family='Old Standard TT, serif',
            size=14,
            color='black'
        ),
        exponentformat='e',
        showexponent='all'
    )
)
fig = go.Figure(data=[traceNormalizedBreathing,traceDerivativeBreathing,traceNewtonRootBreathing,traceBisectionRootBreathing], layout=layout)
iplot(fig, filename='axes-labels')


**Reflection points**
- Which method Newton's or Bisection is better? Why?
- Choose other initial values and find all possible roots

**Exercise Set #2**

1. Write a loop to find all the roots of the breathing function using the Bisection method
2. Write a loop to find all the roots of the breathing function using Newton's method
3. Visualize all the roots in a graph
3. Write a function to calculate the **breathing rate** which is equal to number of roots divided by time taken
